In [0]:
import numpy as np
import pandas as pd
from keras import models
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# import all necessary libraries which are going to be use in binary classification

In [0]:
seed = 7
np.random.seed(seed)


In [4]:
import io 
from google.colab import files
dataFile = files.upload()
df = pd.read_csv(io.StringIO(dataFile['sonar.csv'].decode('utf-8')))
df
dataset = df.values
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]
print(X)
print(Y)

Saving sonar.csv to sonar.csv
[[0.0453 0.0523 0.0843 ... 0.0049 0.0052 0.0044]
 [0.0262 0.0582 0.1099 ... 0.0164 0.0095 0.0078]
 [0.01   0.0171 0.0623 ... 0.0044 0.004  0.0117]
 ...
 [0.0522 0.0437 0.018  ... 0.0138 0.0077 0.0031]
 [0.0303 0.0353 0.049  ... 0.0079 0.0036 0.0048]
 [0.026  0.0363 0.0136 ... 0.0036 0.0061 0.0115]]
['R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 

In [5]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# 1 is for Rock 
# 0 is for Mine
print(encoded_Y)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [0]:
def create_baseline():
    #create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [13]:

# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 82.09% (7.59%)


In [8]:
# evaluate baseline model with standardized dataset
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 85.54% (4.17%)


In [9]:
def create_smaller():
    # create model
    model = Sequential()
    model.add(Dense(30, input_dim=60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 85.52% (5.23%)


In [10]:
# larger model
def create_larger():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 86.97% (4.75%)


In [15]:
#added_more_layers
def add_layers():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, kernel_initializer='normal', activation='relu'))
    #added more layers
    model.add(Dense(30, kernel_initializer='normal', activation='relu'))
    model.add(Dense(15, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=add_layers, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("add_layers: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

add_layers: 87.02% (5.66%)


In [16]:
#added_layers_units
def add_layer_units():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(80, kernel_initializer='normal', activation='relu'))
    #making layers unit bigger
    model.add(Dense(80, kernel_initializer='normal', activation='relu'))
    model.add(Dense(35, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=add_layer_units, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("add_layer_units: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

add_layer_units: 85.02% (5.76%)


In [17]:
# added_more_epochs
def more_epochs():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, kernel_initializer='normal', activation='relu'))
    #added more layers
    model.add(Dense(30, kernel_initializer='normal', activation='relu'))
    model.add(Dense(15, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
#train data on more epochs 
estimators.append(('mlp', KerasClassifier(build_fn=more_epochs, epochs=200, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("more_epochs: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

more_epochs: 83.59% (5.70%)


In [18]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.utils import to_categorical

# By using functional API
def funcational_API():
    # Multilayer Perceptron
    visible = Input(shape=(60,))
    hidden1 = Dense(80, activation='relu')(visible)
    hidden2 = Dense(75, activation='relu')(hidden1)
    hidden3 = Dense(70, activation='relu')(hidden2)
    output = Dense(1, activation='sigmoid')(hidden3)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['accuracy'])
    model.fit(X, encoded_Y, epochs=10, batch_size=20)
    # summarize layers
    #print(model.summary())
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
#train data on more epochs 
estimators.append(('mlp', KerasClassifier(build_fn=funcational_API, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Functional_API: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/10
207/207 [==============================] - 6s 30ms/step - loss: 0.6809 - acc: 0.5700
Epoch 2/10
207/207 [==============================] - 0s 446us/step - loss: 0.6428 - acc: 0.6425
Epoch 3/10
207/207 [==============================] - 0s 453us/step - loss: 0.6136 - acc: 0.6812
Epoch 4/10
207/207 [==============================] - 0s 451us/step - loss: 0.5606 - acc: 0.7585
Epoch 5/10
207/207 [==============================] - 0s 482us/step - loss: 0.5180 - acc: 0.7585
Epoch 6/10
207/207 [==============================] - 0s 450us/step - loss: 0.4951 - acc: 0.7633
Epoch 7/10
207/207 [==============================] - 0s 480us/step - loss: 0.4669 - acc: 0.8116
Epoch 8/10
207/207 [==============================] - 0s 452us/step - loss: 0.4313 - acc: 0.8116
Epoch 9/10
207/207 [==============================] - 0s 428us/step - loss: 0.4015 - acc: 0.8164
Epoch 10/10
207/207 [==============================] - 0s 417us/step - loss: 0.3878 - acc: 0.8309
Epoch 1/10
207/207 [==========

207/207 [==============================] - 0s 435us/step - loss: 0.5526 - acc: 0.7101
Epoch 6/10
207/207 [==============================] - 0s 449us/step - loss: 0.5118 - acc: 0.7246
Epoch 7/10
207/207 [==============================] - 0s 446us/step - loss: 0.4635 - acc: 0.7778
Epoch 8/10
207/207 [==============================] - 0s 482us/step - loss: 0.4681 - acc: 0.7729
Epoch 9/10
207/207 [==============================] - 0s 436us/step - loss: 0.4861 - acc: 0.7343
Epoch 10/10
207/207 [==============================] - 0s 431us/step - loss: 0.4226 - acc: 0.8019
Epoch 1/10
207/207 [==============================] - 7s 31ms/step - loss: 0.6804 - acc: 0.5604
Epoch 2/10
207/207 [==============================] - 0s 443us/step - loss: 0.6482 - acc: 0.6957
Epoch 3/10
207/207 [==============================] - 0s 466us/step - loss: 0.6052 - acc: 0.7005
Epoch 4/10
207/207 [==============================] - 0s 472us/step - loss: 0.5582 - acc: 0.7488
Epoch 5/10
207/207 [=====================

Epoch 9/10
207/207 [==============================] - 0s 465us/step - loss: 0.4674 - acc: 0.7778
Epoch 10/10
207/207 [==============================] - 0s 415us/step - loss: 0.4473 - acc: 0.8309
Epoch 1/10
207/207 [==============================] - 7s 32ms/step - loss: 0.6833 - acc: 0.5652
Epoch 2/10
207/207 [==============================] - 0s 414us/step - loss: 0.6472 - acc: 0.6715
Epoch 3/10
207/207 [==============================] - 0s 452us/step - loss: 0.6108 - acc: 0.6860
Epoch 4/10
207/207 [==============================] - 0s 472us/step - loss: 0.5668 - acc: 0.7778
Epoch 5/10
207/207 [==============================] - 0s 450us/step - loss: 0.5348 - acc: 0.7536
Epoch 6/10
207/207 [==============================] - 0s 434us/step - loss: 0.4873 - acc: 0.8019
Epoch 7/10
207/207 [==============================] - 0s 422us/step - loss: 0.4741 - acc: 0.7729
Epoch 8/10
207/207 [==============================] - 0s 474us/step - loss: 0.4498 - acc: 0.8019
Epoch 9/10
207/207 [==========

Epoch 3/10
207/207 [==============================] - 0s 426us/step - loss: 0.5518 - acc: 0.7874
Epoch 4/10
207/207 [==============================] - 0s 408us/step - loss: 0.5295 - acc: 0.7585
Epoch 5/10
207/207 [==============================] - 0s 439us/step - loss: 0.4953 - acc: 0.7778
Epoch 6/10
207/207 [==============================] - 0s 428us/step - loss: 0.4760 - acc: 0.7826
Epoch 7/10
207/207 [==============================] - 0s 411us/step - loss: 0.4565 - acc: 0.7778
Epoch 8/10
207/207 [==============================] - 0s 420us/step - loss: 0.4237 - acc: 0.8213
Epoch 9/10
207/207 [==============================] - 0s 465us/step - loss: 0.4034 - acc: 0.8454
Epoch 10/10
207/207 [==============================] - 0s 452us/step - loss: 0.4262 - acc: 0.8164
Epoch 1/10
207/207 [==============================] - 7s 33ms/step - loss: 0.6787 - acc: 0.5797
Epoch 2/10
207/207 [==============================] - 0s 423us/step - loss: 0.6426 - acc: 0.6377
Epoch 3/10
207/207 [==========

207/207 [==============================] - 0s 446us/step - loss: 0.4992 - acc: 0.7826
Epoch 8/10
207/207 [==============================] - 0s 429us/step - loss: 0.4698 - acc: 0.7633
Epoch 9/10
207/207 [==============================] - 0s 460us/step - loss: 0.4367 - acc: 0.8213
Epoch 10/10
207/207 [==============================] - 0s 408us/step - loss: 0.4062 - acc: 0.8164
Functional_API: 86.97% (6.76%)


In [29]:
import tensorflow as tf
import keras
class MyModel(tf.keras.Model):
    
        def __init__(self):
            super(MyModel, self).__init__()
            self.dense1= Dense(70, activation='relu')
            self.dense2= Dense(50, activation='relu')
            self.dense3 = Dense(1, activation='sigmoid')

        def call(self, inputs):
            x = self.dense1(inputs)
            x = self.dense2(x)
            return self.dense3(x)
                        
                    
model = MyModel()
model.compile(loss='categorical_crossentropy', optimizer='adam',   metrics=['accuracy'])
history=model.fit(X,encoded_Y, epochs=100,batch_size=50,verbose=False)
history_dict = history.history
history_dict.keys()
acc_values = history_dict['acc']
print("Result: %.2f%% (%.2f%%)" % (np.mean(acc_values)*100, np.std(acc_values)*100))



Result: 53.62% (0.00%)


In [8]:
from keras import models
from keras import layers
def build_model():
    # create model
    model = Sequential()
    model.add(Dense(100, input_dim=60, kernel_initializer='normal' ,activation='relu'))
    model.add(Dense(70,  kernel_initializer='normal' ,activation='relu'))
    model.add(Dense(1,kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
k=10
num_val_samples = len(X) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold #', i)
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = encoded_Y[i * num_val_samples: (i + 1) * num_val_samples]
#     print(val_data,val_targets)
    partial_train_data =np.concatenate([X[:i * num_val_samples],X[(i + 1) * num_val_samples:]],axis=0)
    partial_train_targets =np.concatenate([encoded_Y[:i * num_val_samples],encoded_Y[(i + 1) * num_val_samples:]],axis=0)
    model = build_model()
    model.fit(partial_train_data, partial_train_targets,epochs=num_epochs, batch_size=5, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)
print("Result: %.2f%% (%.2f%%)" % (np.mean(all_scores)*100, np.std(all_scores)*100))


processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9
Result: 62.50% (17.50%)


In [32]:
# Data Normalizing
mean=X.mean(axis=0)
X-=mean
std=X.std(axis=0)
X/=std

In [35]:
from keras import models
from keras import layers
def build_model():
    # create model
    model = Sequential()
    model.add(Dense(100, input_dim=60, kernel_initializer='normal' ,activation='relu'))
    model.add(Dense(70,  kernel_initializer='normal' ,activation='relu'))
    model.add(Dense(1,kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
k=10
num_val_samples = len(X) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold #', i)
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = encoded_Y[i * num_val_samples: (i + 1) * num_val_samples]
#     print(val_data,val_targets)
    partial_train_data = np.concatenate([X[:i * num_val_samples],X[(i + 1) * num_val_samples:]],axis=0)
    partial_train_targets = np.concatenate([encoded_Y[:i * num_val_samples],encoded_Y[(i + 1) * num_val_samples:]],axis=0)
    model = build_model()
    model.fit(partial_train_data, partial_train_targets,epochs=num_epochs, batch_size=5, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)
print("Result: %.2f%% (%.2f%%)" % (np.mean(all_scores)*100, np.std(all_scores)*100))

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9
Result: 65.00% (17.61%)
